# Train 4 and 2 different models on the data with players separated by positions
4 models where players are separated by positions (GK, DEF, MID, FWD)
2 models where players are separated by positions (GK, FIELD_PLAYERS)